In [ ]:
from z3 import *

## Factory Production - Z3 Optimizer

You have a limited amount of labor hours, machine time and raw material.
Maximize the profit given the following constraints.

You have 500 labor hours, 800 machine hours and 600 units of material.

|Name|ProductID|Profit|Labor Hours|Machine Time|Raw Materials|
|---|---|---|---|---|---|
|Phone Case|A|10|3|3|4|
|Phone Charger|B|30|5|3|2|
|Smarphone|C|50|4|5|6|


This can be expressed as a **linear programming** problem.  

$$
\begin{align}
\max f(x) = 10 * A + 30 * B + 50 * C \\
\text{with contraints}\\
3*A + 5*B+ 4*C <= 500 \\
3*A + 3*B + 5*C <= 800 \\
4*A + 2*B + 6*C <= 600 \\
A>=0 \\
B>=0 \\
C>=0
\end{align}
$$

In [ ]:
optimizer = Optimize() # Create a Z3 optimize environment

# Define Variables
phone_cases = Int('A') # Number of phone cases produced
phone_charger = Int('B') # Phone chargers
smartphones = Int('C') # Smartphones

# Labor hours
optimizer.add(3*phone_cases + 5*phone_charger+ 4*smartphones <= 500)

# Machine time
optimizer.add(3*phone_cases + 3*phone_charger + 5*smartphones <= 800)

# Raw materials
optimizer.add(4*phone_cases + 2*phone_charger + 6*smartphones <= 600)

# Non-negative restrictions
optimizer.add([phone_cases>=0, phone_charger>=0, smartphones>=0])


# Define the objective function
profit = 10 * phone_cases + 30 * phone_charger + 50 * smartphones

# Maximize the objective function
optimizer.maximize(profit)

# Check if the problem can be solved
if optimizer.check() == sat:
    model = optimizer.model()
    print("Optimal production plan:")
    print(f"Produce {model.evaluate(phone_cases)} cases.")
    print(f"Produce {model.evaluate(phone_charger)} chargers.")
    print(f"Produce {model.evaluate(smartphones)} smartphones.")
    print(f"Maximum Profit: ${model.evaluate(profit)}")
else:
    print("No feasible production plan found.")


After showing this new plan to the sales team, they add a new constraint. If we produce more than 50 Smartphones, then we should produce at least enough phone cases for 30% of those.

In [ ]:
optimizer.push()
optimizer.add(If(smartphones > 50, phone_cases >= smartphones * 0.3, True))

# Check if the problem can be solved
if optimizer.check() == sat:
    model = optimizer.model()
    print("Optimal production plan:")
    print(f"Produce {model.evaluate(phone_cases)} cases.")
    print(f"Produce {model.evaluate(phone_charger)} chargers.")
    print(f"Produce {model.evaluate(smartphones)} smartphones.")
    print(f"Maximum Profit: ${model.evaluate(profit)}")
else:
    print("No feasible production plan found.")
optimizer.pop()


## Dependency Chaos - Z3 Solver

As part of the production line, you need to manage different parts and chips that are used in different devices.
![title](Images/deps.png)

In [ ]:
## Generate Constraint for Package dependency
def DependsOn(package, deps):
    return And( [ Implies(package, dep) for dep in deps ] )

# Generate constraint for conflicting packages
def Conflict(p1, p2):
    return Or(Not(p1), Not(p2))

In [ ]:
b, c, d, e, f, g = Bools('b c d e f g')
smartphone = BoolVal(True)
charger = BoolVal(True)
s = Solver()

constraints = [
    DependsOn(smartphone, [b, c]),
    DependsOn(charger, [b,f]),
    DependsOn(b, [d]),
    DependsOn(c, [Or(d, e), Or(f, g)]),
    Conflict(d, e),
    Conflict(f, g)
    ]

s.add(constraints)

if s.check() == sat:
    print(s.model())

## Code Verification - Z3 proof
The OS for the OnePluZ3 is written in C. While doing a code review of a coworkers code, you notice a strange C function.
```c
// Magic function
uint32_t f(int32_t v) {    
    int32_t const mask = v >> 31;
    uint32_t r = (v + mask) ^ mask;
    return r;
}
```
Unfortunately, your coworker left for their holiday and is unable to explain what this function is supposed to do.  
You try to figure out what it does by modeling it in python:

```
Speaker notes:
Notice how you can use the function normally in python, but you can also give a Z3 expression as parameter. This is the power of operator overloading.
```

In [ ]:
def f(v):
    mask = v >> 31;
    r = (v + mask) ^ mask
    return r

In [ ]:
print(f(0)) # => 0
print(f(1)) # => 1
print(f(5)) # => 5
print(f(100)) # => 100
print(f(1000)) # => 1000

The function returns its input value? Something seems fishy, let's use Z3 to proof our theory.

In [ ]:
x = BitVec("x",32)
y = f(x)
prove(x == y)

Z3 interprets bitvectors as unsigned integers by default.
The counterexample is outside of the range of unsigned integers (max 2147483647).

In [ ]:
import numpy as np
# Interpret as 32 bit unsigned, then reinterpret those bits as signed.
print(np.int32(np.uint32(2717999451)))

In [ ]:
print(f(-1576967845)) # => 1576967845
print(f(-5)) # => 5
print(f(-100)) # => 100

This function appears to return the absolute value of an integer. But you are still sceptical. How can you be sure that this is what this function does? And maybe even more importantly, that it won't break for a very specific input value?

Let's try to prove our new theory.

In [ ]:
x = BitVec("x",32)
y = f(x)
#prove(If(x >= 0,y == x, y == -x))
prove(If(x >= 0,y == x, y == -x), show=True)

```
Speaker notes:
In Z3 a proof is an exhaustive search without finding a counter example.
Switch commented lines just above. Show how the proof works.
```

## Optimizing dinner with Z3
To reward you for your great work, management decides to treat your team to a nice dinner. However, you have a very strict budget allocated for every minute detail of your meal. They even gave you an upper limit for much you can spend on appetizers!  
In an attempt to get every single cent from management, the following scene unfolds:
![title](https://imgs.xkcd.com/comics/np_complete.png)

[Source - xkcd 287](https://www.explainxkcd.com/wiki/index.php/287:_NP-Complete)

This problem is a variation of the knapsack problem and NP-complete.
You decide to help the waiter out and pull out your laptop:

In [ ]:
appetizers = ["Mixed Fruit", "French Fries", "Side Salad", "Hot Wings", "Mozzarella Sticks", "Sampler Plate"]
prices = [2.15, 2.75, 3.35, 3.55, 4.20, 5.80]

a = IntVector('a', 6) # Int vector with 6 variables
s = Solver()

price_products = [price * var for price,var in zip(prices,a)]
total = sum(price_products)

#print(total)

s.add(total == 15.05)

for i in range(6):
    s.add(a[i] >= 0)

s.add(a[0] != 7)
if s.check() == sat:
    m = s.model()
    for i in range(6):
        print(m.evaluate(a[i]),appetizers[i])
    print()


```
Speaker Notes:
a coworker pipes up, they don't want Mixed Fruit, add another restriction

Show total as Z3 interprets it
```

## Sudoku

During the dinner, the conversation shifts from work to leisure and someone mentions a challenging Sudoku puzzle they've been struggling with. You decide to demonstrate how versatile Z3 can be. It can even be used to solve logic puzzles.
```
Speaker Notes: Switch to sudoku solver notebook
```

## Z3 and Philosophy

You somehow land in a philosophical mood. #todo Come up with a better story.

Axioms:
1. All humans are mortal.
2. Socrates is a man.  

Conclusion :
- Socrates is mortal



In [ ]:
Object = DeclareSort('Object') # uninterpreted sort

Human = Function('Human', Object, BoolSort())
Mortal = Function('Mortal', Object, BoolSort())

socrates = Const('socrates', Object)

x = Const('x', Object)

axioms = [
    ForAll([x], Implies(Human(x), Mortal(x))),
    Human(socrates)
]


s = Solver()
s.add(axioms)

print(s.check()) # sat => axioms are coherent

In [ ]:
# classical refutation
s.add(Not(Mortal(socrates))) # Socrates is not mortal

print(s.check()) # prints unsat so socrates is Mortal


Speaker Notes:  
Covers:
- Uninterpreted/free Sorts : Sorts with no a-priori interpretation
- uninterpreted/free functions
- Quantifiers
  
[Link](https://microsoft.github.io/z3guide/programming/Z3%20Python%20-%20Readonly/advanced#uninterpreted-sorts)

## Simpsons



### Entailment

Facts:
- Marge is parent of Lisa. : `ParentOf(Marge,Lisa)`
- Homer is parent of Lisa. : `ParentOf(Homer,Lisa)`
- Homer is the father of Bart. : `hasFather(Bart) = Homer`

Rules:

![img](Images/father_parent.png)



![img](Images/parent_sibling.png)

Prove that Lisa and Bart are siblings.


In [ ]:
Person = DeclareSort('Person') # uninterpreted sort

homer, marge, bart, lisa = Consts("homer marge bart lisa", Person)
hasFather = Function("hasFather", Person, Person)
parentOf = Function("parentOf", Person, Person, BoolSort())
siblings = Function("siblings", Person, Person, BoolSort())

x = Const('x', Person)
y = Const('y', Person)
z = Const('z', Person)

axioms = [
    parentOf(marge,lisa),
    parentOf(homer,lisa),
    hasFather(bart) == homer,
]
rules = [
    ForAll([x,y], Implies(hasFather(x) == y, parentOf(y,x))),
    ForAll([x,y,z], 
        Implies(
            And(parentOf(x,y), parentOf(x,z)),
            siblings(y,z)
    )),
]


s = Solver()
s.add(axioms)
s.add(rules)

#s.add(Not(siblings(bart,lisa)))

print(s.check())